In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from Model import *
import tensorflow_addons as tfa
from sklearn.metrics import classification_report

In [2]:
train = pd.read_csv("../../dataset/goodreads_train.csv")
test = pd.read_csv("../../dataset/goodreads_test.csv")
vocabulary2 = np.load('../../vocabulaires/voc_without_std_word_count_5.npy', allow_pickle=True)

In [6]:
# show comment with 0 votes
train[train['n_comments'] == 0]

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
13,8842281e1d1347389f2ab93d60773d4d,16158596,6ff8bbc4856aa403bbd8990407c9c77a,0,Recommended by David Risher,Mon Jul 07 10:56:15 -0700 2014,Mon Jul 07 10:56:39 -0700 2014,NaN,NaN,0,0
16,8842281e1d1347389f2ab93d60773d4d,17860739,022bb6daffa49adc27f6b20b6ebeb37d,4,An amazing and unique creation: JJ Abrams and ...,Wed Mar 26 13:51:30 -0700 2014,Tue Sep 23 01:44:36 -0700 2014,Sun Sep 21 00:00:00 -0700 2014,Sat Jul 26 00:00:00 -0700 2014,7,0
21,8842281e1d1347389f2ab93d60773d4d,12953520,8666dfd554e310084c4bc72b2d357f3d,4,A hilarious romp through the known and unknown...,Tue Jul 10 20:26:21 -0700 2012,Wed Sep 12 23:32:04 -0700 2012,Mon Sep 10 00:00:00 -0700 2012,Tue Sep 04 00:00:00 -0700 2012,1,0
...,...,...,...,...,...,...,...,...,...,...,...
899994,b9450d1c1f97f891c392b1105959b56e,12991245,ae4232f6979aa78c54cc3111ee1b9dc5,5,"Whew, this book is freaking brilliant! \n I he...",Mon Nov 19 00:03:10 -0800 2012,Tue Nov 20 10:39:18 -0800 2012,Mon Nov 19 00:00:00 -0800 2012,Mon Nov 19 00:00:00 -0800 2012,3,0
899995,b9450d1c1f97f891c392b1105959b56e,11832081,7c34c15ad8cb28a4afca32946e2016ea,3,3.5 stars. \n Jenna is a popular YA author and...,Fri Nov 16 22:10:17 -0800 2012,Sat Nov 17 22:42:44 -0800 2012,Sat Nov 17 00:00:00 -0800 2012,Fri Nov 16 00:00:00 -0800 2012,0,0
899997,b9450d1c1f97f891c392b1105959b56e,8430896,feb9fe6439f9fbba8a1cce36020ad977,4,** spoiler alert ** \n 3.5 stars. \n This book...,Tue Nov 13 05:38:24 -0800 2012,Tue Nov 13 23:29:17 -0800 2012,Tue Nov 13 00:00:00 -0800 2012,Tue Nov 13 00:00:00 -0800 2012,1,0
899998,b9450d1c1f97f891c392b1105959b56e,12275680,741e29369f70531bfffbd02add86f228,4,** spoiler alert ** \n Another fun read from M...,Mon Nov 12 05:42:30 -0800 2012,Tue Nov 13 22:32:05 -0800 2012,Tue Nov 13 00:00:00 -0800 2012,Mon Nov 12 00:00:00 -0800 2012,0,0


In [7]:
train[train['n_comments'] > 0]

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1
5,8842281e1d1347389f2ab93d60773d4d,19398490,ea4a220b10e6b5c796dae0e3b970aff1,4,A beautiful story. It is rare to encounter a b...,Sun Jan 03 21:20:46 -0800 2016,Tue Sep 20 23:30:15 -0700 2016,Tue Sep 13 11:51:51 -0700 2016,Sat Aug 20 07:03:03 -0700 2016,35,5
6,8842281e1d1347389f2ab93d60773d4d,22318578,5fe9882bfe4b0520a322820c4c55747d,5,5 stars for giving me a better framework for h...,Sun Jun 07 12:50:13 -0700 2015,Wed Mar 22 11:36:58 -0700 2017,Sun Aug 09 00:00:00 -0700 2015,Sun Jun 07 00:00:00 -0700 2015,24,3
...,...,...,...,...,...,...,...,...,...,...,...
899989,b9450d1c1f97f891c392b1105959b56e,15818969,b591cacf61c6940699d193313a4d77a7,5,** spoiler alert ** \n This is definitely one ...,Sat Nov 24 23:20:58 -0800 2012,Tue Nov 27 05:45:48 -0800 2012,Mon Nov 26 00:00:00 -0800 2012,Sun Nov 25 00:00:00 -0800 2012,6,4
899991,b9450d1c1f97f891c392b1105959b56e,16113791,2a0209aab2627778617e1565e674833b,5,** spoiler alert ** \n 4.5 stars. \n This is t...,Wed Nov 21 22:39:24 -0800 2012,Fri Dec 21 21:34:22 -0800 2012,Sat Dec 15 00:00:00 -0800 2012,Fri Dec 14 00:00:00 -0800 2012,1,2
899992,b9450d1c1f97f891c392b1105959b56e,15992339,e4d6c4c063cbbe442b5adaa6e57aaa05,3,This is a short story about Kami's summer befo...,Wed Nov 21 06:05:17 -0800 2012,Thu Nov 22 07:10:05 -0800 2012,Wed Nov 21 00:00:00 -0800 2012,Wed Nov 21 00:00:00 -0800 2012,0,3
899993,b9450d1c1f97f891c392b1105959b56e,15717943,813928f3116d2a8da6949c8c6095af8f,5,** spoiler alert ** \n 5 Stars! \n Thank you M...,Mon Nov 19 06:10:32 -0800 2012,Fri Dec 21 22:11:53 -0800 2012,Thu Dec 20 00:00:00 -0800 2012,Wed Dec 19 00:00:00 -0800 2012,4,6


In [3]:
# lenght = 0
# list_word = []
# list_weight = []
# with open('../dataset/glove.6B.300d.txt', encoding='utf-8') as f:
#     for line in tqdm(f):
#         values = line.split()
#         word = values[0]#.replace('<','').replace('>', '')
#         coefs = values[1:]
#         if len(coefs) == 300:
#             #list_weight.append(coefs)
#             list_word.append(word)
#             lenght += 1
#         else:
#             print("incorect dim")

In [4]:
#model_list = [cnn1,cnn2, cnn3,cnn4,cnn5,cnn6, cnn8, cnn9, cnn10]
model_list = [unet1]
def scheduler(epoch, lr):
    if epoch < 4:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
for seed in [8,5]:
    keras.utils.set_random_seed(seed)
    for model_obj in model_list:
        model = model_obj.Model(vocabulary2)
        model.model.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.001),
                           loss=keras.losses.categorical_crossentropy,
                           metrics=[keras.metrics.categorical_accuracy, tfa.metrics.F1Score(num_classes=6, average='weighted')]
                           )
        print(model.name)
        #print(model.model.summary())
        if not os.path.exists(f"logs/{model.name}"):
            os.mkdir(f"logs/{model.name}")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs/{model.name}")
        model.run_experiment([train['review_text'], train['n_comments'], train['n_votes']], train['rating'], epochs=9, callbacks=[keras.callbacks.LearningRateScheduler(scheduler,0)], batch_size=60, validation_split=0.2)

unet1
Epoch 1/9
12000/12000 [==============================] - 212s 17ms/step - loss: 1.1997 - categorical_accuracy: 0.4869 - f1_score: 0.4671 - val_loss: 1.0371 - val_categorical_accuracy: 0.5542 - val_f1_score: 0.5336 - lr: 0.0010
Epoch 2/9
12000/12000 [==============================] - 207s 17ms/step - loss: 1.0472 - categorical_accuracy: 0.5552 - f1_score: 0.5444 - val_loss: 1.0000 - val_categorical_accuracy: 0.5718 - val_f1_score: 0.5622 - lr: 0.0010
Epoch 3/9
12000/12000 [==============================] - 205s 17ms/step - loss: 1.0072 - categorical_accuracy: 0.5737 - f1_score: 0.5660 - val_loss: 0.9846 - val_categorical_accuracy: 0.5775 - val_f1_score: 0.5686 - lr: 0.0010
Epoch 4/9
12000/12000 [==============================] - 208s 17ms/step - loss: 0.9804 - categorical_accuracy: 0.5863 - f1_score: 0.5801 - val_loss: 0.9737 - val_categorical_accuracy: 0.5835 - val_f1_score: 0.5768 - lr: 0.0010
Epoch 5/9
12000/12000 [==============================] - 210s 18ms/step - loss: 0.9548

In [5]:
res = model.model.predict([train['review_text'], train['n_comments'], train['n_votes']])

28125/28125 [==============================] - 164s 6ms/step


In [ ]:
restest = model.model.predict([test['review_text'], test['n_comments'], test['n_votes']])

In [ ]:
ff = []
for line in tqdm(restest):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
test_data = np.array(ff)

ff = []
for line in tqdm(res):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
train_data = np.array(ff)

In [ ]:

print(classification_report(train['rating'], train_data))

In [ ]:
test['rating'] = test_data

id = test['review_id'].to_numpy()
rating = test['rating'].to_numpy()
df = pd.DataFrame( columns=['review_id', 'rating'])
df['review_id'] = id
df['rating'] = rating

In [ ]:
df.to_csv('submission_unet1_embedding_class_weights_model.csv',index=False )
